https://arxiv.org/abs/2505.22181

symbolic execution of ground orders.

Making a z3 problem out of a given set of _term_ rewrites. A kbo can always oreitn any given ground eqauytions, it's only the partiality of nonground that makes that problem interesting

1. Find parameters of order for gvien Rewrite system
2. Find oreintation and parameter of equation system

Kind termination or of ground rewrite system. That should be easy. Solving for the wegiths


WPO
(monotonic) semantic path ordering https://static.aminer.org/pdf/PDF/000/071/240/complete_monotonic_semantic_path_orderings.pdf
https://www.cs.tau.ac.il/~nachumd/term/L8.pdf
https://arxiv.org/abs/2307.13973 generalizing weighted path ordering

https://www.cs.tau.ac.il/%7Enachumd/term/

These

Cody questions KBO = size?

What is KBO for applicative terms. LFHOL @(@(f, arg1), args2)
Hmm. Well it is kind of inverted from what you'd want. You need to dig down to see the flat stuff first


In [14]:
from kdrag.all import *

from typing import Callable

from dataclasses import dataclass

@dataclass(frozen=True)
class App:
    f : str
    args : tuple["App", ...]

    def symbols(self) -> set[str]:
        return {self.f}.union(*(a.symbols() for a in self.args))

    def size(self, W : Callable[[str], int]) -> int:
        return W(self.f) + sum(a.size(W) for a in self.args)
    """
    def kbo(self, t, W): # coulld also take in precedence
        size1, sizet = self.size(W), t.size(W)
        if size1 == sizet:
        else:
            return size1 > size2
    """

    def sym_kbo(self, t, W):
        # t and self are not symbolic, which makes it easier
        # If they _are_, which is kind of the case for ordering rewriting with variables...
        size1, sizet = self.size(W), t.size(W)
        if t.f == self.f:
            undef = smt.FreshConst(smt.BoolSort(), prefix="undef") # won't be used. The assert False
            for a, b in zip(self.args, t.args):
                if a != b: # This is where it'll happen
                    res = a.kbo(b, W)
                    break
                else:
                    assert False
            return smt.If(size1 == sizet, res, size1 > sizet)
        else:
            return smt.If(size1 == sizet, self.f > t.f, size1 > sizet)

def W(f : str) -> smt.ArithRef:
    return smt.Int(f)
def Prec(f):
    return smt.Int("prec_" + f) # Prec(f) > Prec(g) would be a way to achieve a total order (lots of reundancy but whatev)

f = lambda x: App("f", (x,))
a = App("a", ())
b = App("b", ())
g = lambda x: App("g", (x,))
R = [
     (b , f(f(f(a)))),
     #(a, f(a)),
     (g(a), f(b)),
     #(a, b)
]
s = smt.Solver()
for lhs,rhs in R:
    s.add(lhs.sym_kbo(rhs, W))
    for f in lhs.symbols() | rhs.symbols():
        s.add(W(f) > 0)
s.check()
s.model()


[a = 1, f = 1, g = 6, b = 5]

Are there ground rule systems that can't be shown terminating by a KBO? Probs. Uhhh. Hmm.

